In [ ]:
%cd ~/Dropbox/CanvasHacks

#Plotting 
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf')


import pandas as pd
pd.options.display.max_rows = 999

import datetime

from CanvasHacks import environment
from CanvasHacks.Api.RequestTools import *
from CanvasHacks.Api.UrlTools import *
from CanvasHacks.Configuration import InteractiveConfiguration
# import CanvasHacks.GradingTools as GT
# import CanvasHacks.DownloadProcessingTools as PT

# File system
from CanvasHacks.TimeTools import getDateForMakingFileName
from CanvasHacks.Files.FileTools import  create_folder, makeDataFileIterator
from CanvasHacks.Files.JournalsFileTools import get_journal_folders, make_folder_list, calculate_journal_counts
from CanvasHacks.Files.QuizReportFileTools import sort_frames_by_age, get_newest_data

# from CanvasHacks.JournalsFileTools import journal_folder_name, create_folder
# from CanvasHacks.FileTools import getDateForMakingFileName

# Canvas api
from canvasapi import Canvas
from canvasapi.quiz import QuizReport, Quiz
from canvasapi.requester import Requester
from canvasapi.conversation import Conversation

# Initialize a Canvas api objects
canvas = Canvas(environment.CONFIG.canvas_url_base, environment.CONFIG.canvas_token)
requester = Requester(environment.CONFIG.canvas_url_base, environment.CONFIG.canvas_token)

# Configuration
from CanvasHacks.Definitions.skaa import Review, InitialWork, MetaReview
from CanvasHacks.Definitions.unit import Unit #Assignment

# Exceptions
# from CanvasHacks.Errors.review_associations import AlreadyAssigned, SubmissionIncomplete

# Models
from CanvasHacks.Models.student import Student
from CanvasHacks.Models.student import student_from_canvas_user, ensure_student

# Repos
from CanvasHacks.Repositories.DataManagement import DataStore
from CanvasHacks.Repositories.quizzes import QuizRepository, ReviewRepository
from CanvasHacks.Repositories.codes import AccessCodeRepo
from CanvasHacks.Repositories.reviewer_associations import assign_reviewers, AssociationRepository
from CanvasHacks.Repositories.students import StudentRepository

# Storage
from CanvasHacks.DAOs.sqlite_dao import SqliteDAO

# Widgets
from CanvasHacks.Widgets.ConsolidatedTextOutput import make_assignment_header, make_consolidated_text_fields
from CanvasHacks.Widgets.InputFields import make_course_ids_input, make_canvas_token_input, make_canvas_url_input, make_general_reset_button
from CanvasHacks.Widgets.AssignmentSelection import make_assignment_chooser, view_selected_assignments, view_ungraded_assignments
from CanvasHacks.Widgets.LiveSelection import make_test_selector
from CanvasHacks.Widgets.AssignmentSelection import make_unit_chooser

# Plotting
from CanvasHacks.Text.VisualizationTools import rotate_x_labels
# from CanvasHacks.Repositories.quizzes import fix_forgot_answers

import inspect
def look_inside(obj):
    print(inspect.getmembers(obj, lambda a:not(inspect.isroutine(a))))
    
    
SEMESTER_NAME = 'S20'
LOC = '{}/Box Sync/TEACHING/Phil 305 Business ethics/Phil305 S20'.format(environment.ROOT)# placeholder for where the access codes are stored
ACCESS_CODES_FP = "{}/{}-assignment-access-codes.xlsx".format(LOC, SEMESTER_NAME)
    
    
# LIKERT_PLOT_ORDER = ['Forgot', 'Strongly disagree', 'Disagree', 'Agree', 'Strongly agree']
# LIKERT_NUM_MAP = {'Forgot' : 0, 'Strongly disagree': 1, 'Disagree': 2, 'Agree': 3, 'Strongly agree': 4}


In [ ]:
make_test_selector()
make_unit_chooser()

In [ ]:
environment.CONFIG.course_ids[0]

# Word count of assignments vs historical

In [ ]:
# acquire, clean, and store text

In [ ]:
from CanvasHacks.Assessment.api import get_all_essay_assigns_for_class, store_course_essays
courses_to_get = [  ]

# for cid in courses_to_get:
#     store_course_journals(cid, 'S19')

# es = get_all_essay_assigns_for_class(environment.CONFIG.course_ids[0], term='S20')
# es
# for cid in environment.CONFIG.course_ids:
# store_course_essays(environment.CONFIG.course_ids[0], 'S20', start_week=12)

for cid in environment.CONFIG.course_ids:
    store_course_essays(cid, 'S20', start_unit=1)

## Process and store wordbags

In [ ]:
from CanvasHacks.Assessment.processing import process_essay_entries
from CanvasHacks.Assessment.files import EssayFiles

file_handler = EssayFiles()

In [ ]:
fiter = filename_handler.make_content_file_iterator()
existing = filename_handler.bag_files
# makeDataFileList(filename_maker.bag_folder_root)

while True:
    with open(next(fiter), 'r') as f:
        print("Processing ", f.name.split('/')[-1:])
        entries = json.load(f)
        process_essay_entries(entries, existing)

## Load bags

In [ ]:
from CanvasHacks.Assessment.store import load_stored_bags, EssayAssignment, JournalAssignment, TermUnitStore, TermWeekStore, TokenFiltrationMixin


In [ ]:

# def load_stored_bags(file_handler):
#     fiter = file_handler.make_bag_file_iterator()
#     data = []
    
#     if isinstance(file_handler, EssayFiles):
#         AssignmentObj = EssayAssignment
#         ComboObj = TermUnitStore
#     elif isinstance(file_handler, JournalFiles):
#         AssignmentObj = JournalAssignment(**d)
#         ComboObj = TermWeekStore

#     try:
#         while True:
#             with open(next(fiter), 'r') as f:
#                 d = json.load(f)
#                 o = AssignmentObj(**d)
#                 data.append(o)

#     except StopIteration:
#         print("Loaded {} files".format(len(data)))

#     terms = list(set([e.term for e in data]))
    
#     try:
#         divs = list(set([e.unit_number for e in data]))
#     except NameError:
#         divs = list(set([e.week_num for e in data]))

#     stores = []
#     for t in terms:
#         for w in divs:
#             stores.append(ComboObj(t, w, data))

# #     len(week_stores)
    
#     return stores, terms, divs

In [ ]:
stores, terms, units = load_stored_bags(file_handler)

word_counts = pd.DataFrame([{'unit' : s.unit, 'word_count': b} for s in stores for b in s.bag_word_counts])
# word_counts.set_index('unit', inplace=True)

In [ ]:
sns.violinplot(data=word_counts, y='word_count', x='unit')

In [ ]:
word_counts[word_counts.unit == 3]

In [ ]:
word_counts.set_index('unit', inplace=True)

In [ ]:
# sns.kdeplot(word_counts[word_counts.unit == 2], color='r', label='unit 2')
for i in range(1, 5):
    sns.kdeplot(word_counts.loc[i].word_count, label='unit {}'.format(i))
plt.legend()

In [ ]:
word_counts.loc[3]

# Comparisons of student responses

In [ ]:
studentRepo = StudentRepository(environment.CONFIG.course)
studentRepo.download()


if environment.CONFIG.is_test:
    # testing: in memory db
    dao = SqliteDAO()
    print("Connected to testing db")
else:
    db_filepath = "{}/{}-Unit-{}-discussion-review.db".format( environment.LOG_FOLDER, SEMESTER_NAME, environment.CONFIG.unit_number)
    # real: file db
    dao = SqliteDAO(db_filepath)
    dao.initialize_db_file()
    print("Connected to REAL db")

associationRepo = AssociationRepository(dao, environment.CONFIG.unit.discussion_review)

In [ ]:
reviewRepo = ReviewRepository(environment.CONFIG.unit.review, environment.CONFIG.course)
reviewRepo.data = get_newest_data(environment.CONFIG.unit.review)
reviewRepo.set_question_columns(reviewRepo.data)
reviewRepo._fix_forgot_answers()

# add assessees
reviewRepo.data['assessee_id'] = reviewRepo.data.apply(lambda x: associationRepo.get_assessee(environment.CONFIG.unit.review, x.student_id), axis=1)

reviewRepo.data.dropna(subset=['assessee_id'], inplace=True)

len(reviewRepo.data)

In [ ]:
# reviewRepo.data.assessee_id = reviewRepo.data.assessee_id.astype('int32')

In [ ]:
reviewRepo.data.assessee_id.dtype

In [ ]:

# def fix_forgot_answers(reviewRepo):
#     def r(v):
#         if v == 'They forgot to do this':
#             return 'Forgot'
#         return v

#     for c in reviewRepo.multiple_choice_names:
#         reviewRepo.data[c] = reviewRepo.data.apply(lambda x: r(x[c]), axis=1)

# def rotate_x_labels(axis, degrees=45):
#     for tick in axis.get_xticklabels():
#         tick.set_rotation(degrees)

In [ ]:

# fix_forgot_answers(reviewRepo)
d = reviewRepo.data.copy(deep=True)

In [ ]:
d

In [ ]:
rows = nrows=round(len(reviewRepo.multiple_choice_names)/2)
fig, axes = plt.subplots(ncols=2, nrows=rows, figsize=(12,20))

row=0; col=0
for c in reviewRepo.multiple_choice_names:
    title = c.split(':')[1][:65]
    g = sns.countplot(d[c], order=environment.LIKERT_PLOT_ORDER, palette='plasma', ax=axes[row, col])
    g.set_xlabel('')
    axes[row, col].set_title(title)
    rotate_x_labels(axes[row, col])
    if col == 1:
        row += 1
        col = 0
    else:
        col += 1

fig.tight_layout()

# How do reviewers and reviewees rate each other

In [ ]:
numd = d.copy(deep=True)


for c in reviewRepo.multiple_choice_names:
    numd[c] = numd.apply(lambda x: environment.LIKERT_NUM_MAP.get(x[c]), axis=1)


In [ ]:
numd.set_index('student_id', inplace=True)

In [ ]:
f = []
for i, row in numd.iterrows():
    f.append( {
        'assessor' : i,
        'assessee' : row.assessee_id,
        'total' : sum(row[reviewRepo.multiple_choice_names])
    })
f = pd.DataFrame(f)

In [ ]:
g = f.copy(deep=True)
g.set_index('assessee', inplace=True)
f.set_index('assessor', inplace=True)

In [ ]:
b=[]
for sid in f.index:
    try:
        # gave, recieved
        gave = f.loc[sid].total.mean()
        recd = g.loc[sid].total.mean()
        b.append({'gave': gave, 'recd': recd, 'gap': gave - recd})
    except KeyError:
        pass
b = pd.DataFrame(b)
len(b)

In [ ]:
b.gap

0 : Gave and received the same

\> 0: Gave a better score than they received

< 0: Received a better score than they gave

In [ ]:
sns.distplot(b.gap.dropna(), rug=True)

In [ ]:
len(b[b.gap >0])

In [ ]:
len(b[b.gap < 0])

In [ ]:
sns.scatterplot(b.gave, b.recd)

In [ ]:
sns.lmplot(x='gave', y='recd', data=b)

In [ ]:
sns.lmplot(y='gave', x='recd', data=b)

# Feedback on course

In [ ]:
make_test_selector()
make_unit_chooser()

In [ ]:
unit.unit_end_survey

In [ ]:

TERM = 'S20'

SURVEY_FOLDER = '/Users/adam/Box Sync/TEACHING/Phil 305 Business ethics/Surveys/{}'.format(TERM)

def get_unit(filename):
    s = filename.split('_')[0][-1 : ]
    return int(s)

fiter = makeDataFileIterator( SURVEY_FOLDER )
report_frames = [ ]
try:
    while True:
        f = next( fiter )
        unit_num = get_unit(f)
        print( "loading: ", f )
        frame = pd.read_csv( f )
        frame['term'] = TERM
        frame['unit'] = unit_num
        # this makes it freak out for some reason
        #         frame.set_index('student_id', inplace=True)
        report_frames.append( frame )
except StopIteration:
    pass

class_data = report_frames[0]
len(class_data)

In [ ]:
class_data

In [ ]:
surveyRepo = ReviewRepository(unit.unit_end_survey, course)
surveyRepo.data = class_data
surveyRepo.set_question_columns(surveyRepo.data)
fix_forgot_answers(surveyRepo)

In [ ]:
TIME_ORDER = ['Less than 1 hour', '1-3 hours', '3-5 hours', '5-7 hours', 'More than 7 hours']

rows = nrows=round(len(surveyRepo.multiple_choice_names)/2)
fig, axes = plt.subplots(ncols=2, nrows=rows, figsize=(12,30))

row=0; col=0
for c in surveyRepo.multiple_choice_names:
    title = c.split(':')[1][:65]
    if c == surveyRepo.multiple_choice_names[0]:
        order = TIME_ORDER
    else:
        order = [l for l in environment.LIKERT_PLOT_ORDER if l != 'Forgot']
        
    g = sns.countplot(surveyRepo.data[c], order=order, palette='plasma', ax=axes[row, col])
    g.set_xlabel('')
    axes[row, col].set_title(title)
    rotate_x_labels(axes[row, col])
    if col == 1:
        row += 1
        col = 0
    else:
        col += 1
fig.tight_layout()

# Attic

In [ ]:
def get_newest_data(activity):
    # get data from newest file
    fiter = makeDataFileIterator( activity.folder_path )
    report_frames = [ ]
    try:
        while True:
            f = next( fiter )
            print( "loading: ", f )
            frame = pd.read_csv( f )
            frame.submitted = pd.to_datetime( frame.submitted )
            if 'student_id' not in frame.index:
                frame.rename( { 'id': 'student_id' }, axis=1, inplace=True )
            # this makes it freak out for some reason
            #         frame.set_index('student_id', inplace=True)
            report_frames.append( frame )
    except StopIteration:
        return sort_frames_by_age( report_frames )[0]
    

In [ ]:
# TEST = False

# environment.CONFIG.set_unit_number(1)

# if TEST:
#     environment.CONFIG.set_test()
# # environment.CONFIG.set_live()

# COURSE_ID = environment.CONFIG.course_ids[0]
# print("Working on course: ", COURSE_ID)


# UNIT_NUMBER = 1

# _initialize based on selection
# todo eventually should be integrated into config
# course = canvas.get_course(COURSE_ID)
# unit = Unit(course, environment.CONFIG.unit)
# codeRepo = AccessCodeRepo(ACCESS_CODES_FP, environment.CONFIG.unit)



# if TEST:
#     # testing: in memory db
#     dao = SqliteDAO()
#     print("Connected to testing db")
# else:
#     db_filepath = "{}/{}-Unit-{}-review-assigns.db".format( environment.LOG_FOLDER, SEMESTER_NAME, environment.CONFIG.unit)
#     # real: file db
#     dao = SqliteDAO(db_filepath)
#     dao.initialize_db_file()
#     print("Connected to REAL db")

# associationRepo = AssociationRepository(dao, unit.review)